In [1]:
from haystack.components.builders import PromptBuilder

from haystack.Concepts.getting_start import prompt_template, question

builder = PromptBuilder(
    template="Hello {{name}}! {{greeting}}",
    required_variables=[]
)

In [2]:
builder

Inputs:
  - greeting: Any
  - name: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str

In [3]:
builder = PromptBuilder(
    template="Hello {{name}}! {{greeting}}",
    required_variables=["name"]
)

In [4]:
builder

Inputs:
  - greeting: Any
  - name: Any
  - template: Optional[str]
  - template_variables: Optional[dict[str, Any]]
Outputs:
  - prompt: str

In [5]:
from haystack.components.builders import PromptBuilder

template = """
Current date is: {% now 'UTC' %}
Thank you for providing the date
Yesterday was: {% now 'UTC' - 'days=1' %}
"""

In [6]:
builder = PromptBuilder(template=template)

In [7]:
result = builder.run()["prompt"]

In [8]:
result

'\nCurrent date is: 2025-12-25 16:12:49\nThank you for providing the date\nYesterday was: 2025-12-24 16:12:49'

In [9]:
builder.run()

{'prompt': '\nCurrent date is: 2025-12-25 16:13:17\nThank you for providing the date\nYesterday was: 2025-12-24 16:13:17'}

In [10]:
from haystack.components.builders import PromptBuilder

In [11]:
template = "Translate the following context to {{ target_language }}. Context: {{ snippet }}; Translation:"
builder = PromptBuilder(template=template)
builder.run(target_language="spanish", snippet="I can't speak spanish")

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


{'prompt': "Translate the following context to spanish. Context: I can't speak spanish; Translation:"}

In [12]:
from haystack import Pipeline, Document
from haystack.utils import Secret
from haystack.utils import Secret
from haystack.utils.hf import HFGenerationAPIType
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator
from haystack.components.builders.prompt_builder import PromptBuilder

/home/eyad/haystack/hastack_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
documents = [Document(content="Joe lives in Berlin"), Document(content="Joe is a software engineer")]

In [14]:
prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{query}}
    \nAnswer:
"""

In [22]:
p = Pipeline()
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
p.add_component("llm", HuggingFaceAPIChatGenerator(api_type=HFGenerationAPIType.SERVERLESS_INFERENCE_API,api_params={"model": "Qwen/Qwen2.5-7B-Instruct", "provider": "together"},
token=Secret.from_token("")))
p.connect('prompt_builder', 'llm')

PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


PipelineConnectError: Cannot connect 'prompt_builder' with 'llm': no matching connections available.
'prompt_builder':
 - prompt: str
'llm':
 - messages: list[ChatMessage] (available)
 - generation_kwargs: Optional[dict[str, Any]] (available)
 - tools: Union[list[Tool], list[Toolset], list[Union[Tool, Toolset]], Toolset] (available)
 - streaming_callback: Union[Callable[], Callable[, Awaitable[]]] (available)

In [17]:
result

'\nCurrent date is: 2025-12-25 16:12:49\nThank you for providing the date\nYesterday was: 2025-12-24 16:12:49'

In [18]:
documents = [
    Document(content="Joe lives in Berlin", meta={"name": "doc1"}), Document(content="Joe is a software engineer", meta={"name": "doc1"})
]

In [19]:
new_template = """
    You are a helpful assistant.
    Given these documents, answer the question.
    Documents:
    {% for doc in documents %}
        Document {{ loop.index }}:
        Document name: {{ doc.meta['name'] }}
        {{ doc.content }}
    {% endfor %}

    Question: {{ query }}
    Answer:
"""

In [21]:
question = "What is the capital of Germany?"
p.run({
    "prompt_builder": {"documents": documents, "query": question, "template": new_template},
})

ValueError: Missing input for component llm: messages